## Com funcionen els models  BERT, RoBERTA, o la seva adaptació al català BERTa?

A continuació es mostra un exemple del funcionament del model BERTa (l'adaptació al català del model RoBERTA), i la tasca *fill-mask*.

---

Frase d'exemple: "Ella és \<mask\>."  
Buscarem les probablitats que retorna el model per les següents paraules:
* actriu
* advocada
* infermera
* doctora
* cambrera
* taxista

---

**Índex**

1. Importació de les llibreries necessàries
2. Carregar el model BERTa
3. Procés de Tokenització
4. Exemple "Ella és \<mask\>"
5. Exemple: "El meu pare és \<mask\>
6. Paraules i subparaules

---

Pàgina del model BERTa:
https://huggingface.co/projecte-aina/roberta-base-ca-v2


### 1. Importar llibreries necessàries

In [1]:
import transformers
import torch
from torch.nn.functional import softmax

# Mostrem les versions utilitzades
print("torch        ", torch.__version__)
print("transformers ", transformers.__version__)

torch         1.12.1
transformers  4.20.1


### 2. Carreguem els models

In [36]:
# Carreguem el tokenizer i el model.
# El primer cop que s'utilitzi el model, es descarregarà de la pàgina oficial i es 
# guardarà a la carpeta local "~/.cache/huggingface/hub"

# https://huggingface.co/projecte-aina/roberta-base-ca-v2
tokenizer = transformers.AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
model = transformers.AutoModelForMaskedLM.from_pretrained("projecte-aina/roberta-base-ca-v2")

print("Mida del vocabulari:", tokenizer.vocab_size)

Mida del vocabulari: 50262


### 3. Tokenització

In [23]:
print(f'{"paraula":<10}{"token_id":>10}')
print('================================')
for paraula in ['Ell', 'Ella', 'és', 'actor', 'actriu', 'advocat', 'advocada', 'infermera', '.']:
    print(f'{paraula:<10}{tokenizer.encode(paraula)[1]:>10}')

paraula     token_id
Ell             7785
Ella           11947
és               423
actor          10740
actriu         13751
advocat        11163
advocada       32308
infermera      26849
.               2462


### 4. Exemple "Ella és \<mask\>."

In [26]:
text = "Ella és <mask>."

# Tokenizació
input_ids = tokenizer(text, return_tensors="pt").input_ids

# Posició de la màscara
mask_index = input_ids.tolist()[0].index(tokenizer.mask_token_id)

# Obtenim els valors del model amb el text tokenitzat
model_logits = model(input_ids).logits
model_probs = softmax(model_logits, dim=2)

# Mostra el text tokenitzat i la mida
print("Text original:                   ", text)
print("Text tokenitzat:                 ", input_ids.tolist())
print("Posició de la màcara:            ", mask_index)
print("Text tokenitzat (dimensions):    ", list(input_ids.size()))

# Mostrem els valors donats per el model
print("Model logits (dimensions):       ", list(model_probs.shape))
print("Model probabilitats (dimensions):", list(model_probs.shape))

# Obtenim els 20 valors més probables i els mostrem per pantalla
values, indices = model_probs[0,mask_index].topk(k=20)

print("\n\nMostrem les 20 paraules més probables:")
print(f'{"paraula":<10}{"token_id":>10}{"prob":>12}')
print('================================')
for prob,token_id in zip(values.tolist(), indices.tolist()):
    print(f"{tokenizer.decode(token_id):<10}{token_id:>10}{prob:>12.2%}")

Text original:                    Ella és <mask>.
Text tokenitzat:                  [[0, 11947, 423, 4, 2462, 2]]
Posició de la màcara:             3
Text tokenitzat (dimensions):     [1, 6]
Model logits (dimensions):        [1, 6, 50262]
Model probabilitats (dimensions): [1, 6, 50262]


Mostrem les 20 paraules més probables:
paraula     token_id        prob
 així           1008       4.94%
 la              331       2.63%
 feliç          9238       2.27%
 una             411       1.96%
...              603       1.70%
 aquí           1811       1.65%
 catalana       1486       1.31%
 russa         13202       1.23%
 de              315       1.16%
 francesa       6230       0.92%
 perfecta      13627       0.92%
 diferent       3074       0.90%
 allà           3537       0.87%
 morta         14812       0.85%
 ella           2984       0.82%
 gran            742       0.81%
.                 68       0.72%
 alemanya       9203       0.72%
 dona           1629       0.72%
 bella      

In [13]:
# Busquem les probabilitats per les paraules buscades
print(f'{"paraula":<10}{"token_id":>10}{"prob":>12}')
print("================================")

for paraula in ["actriu", "advocada", "infermera", "doctora", "cambrera", "taxista"]:
    
    # Obtenim el token ID de la paraula (agafem el 2 element).
    
    # S'ha de vigilar amb aquest mètode ja que algunes paraules es codifiquen amb 
    # més d'un valor, però amb les paraules utilitzades amb l'exemple és correcte.

    # Exemple de tokenització amb paraules d'un sol token ("infermera") i paraula multi-token ("infermer")
    # tokenizer.encode("infermera") => [0, 13751, 2]
    # tokenizer.encode("infermer") => [0, 4139, 4653, 2]
    token_id = tokenizer.encode(paraula)[1]
    
    # Obtenim la probabilitat 
    prob = model_probs[0, mask_index, token_id]
    
    
    # Mostrem els resultats
    print(f"{paraula:<10}{token_id:>10}{prob:>12.4%}")


paraula     token_id        prob
actriu         13751     0.2407%
advocada       32308     0.1001%
infermera      26849     0.4912%
doctora        18530     0.1264%
cambrera       41941     0.1268%
taxista        38757     0.0100%


### 5. Exemple: "El meu pare és \<mask\>

In [35]:
text = "El meu pare és <mask>."
# text = "La meva mare és <mask>."
# text = "<mask><mask><mask> és <mask>."
# text = "<mask> és <mask>."

# Tokenizació
input_ids = tokenizer(text, return_tensors="pt").input_ids

# Posició de la màscara
mask_index = input_ids.tolist()[0].index(tokenizer.mask_token_id)

# Obtenim els valors del model amb el text tokenitzat
model_logits = model(input_ids).logits
model_probs = softmax(model_logits, dim=2)


# Mostra el text tokenitzat i la mida
print("Text original:                   ", text)
print("Text tokenitzat:                 ", input_ids.tolist())
print("Posició de la màcara:            ", mask_index)
print("Text tokenitzat (dimensions):    ", list(input_ids.size()))


# Mostrem els valors donats per el model
print("Model logits (dimensions):       ", list(model_probs.shape))
print("Model probabilitats (dimensions):", list(model_probs.shape))

# Obtenim els 20 valors més probables i els mostrem per pantalla
values, indices = model_probs[0, mask_index].topk(k=20)

print("\n\nMostrem les 20 paraules més probables:")
print(f'{"paraula":<16}{"token_id":>10}{"prob":>12}')
print('================================')
for prob, token_id in zip(values.tolist(), indices.tolist()):
    print(f"{tokenizer.decode(token_id):<16}{token_id:>10}{prob:>12.2%}")

Text original:                    El meu pare és <mask>.
Text tokenitzat:                  [[0, 442, 1685, 2912, 423, 4, 2462, 2]]
Posició de la màcara:             5
Text tokenitzat (dimensions):     [1, 8]
Model logits (dimensions):        [1, 8, 50262]
Model probabilitats (dimensions): [1, 8, 50262]


Mostrem les 20 paraules més probables:
paraula           token_id        prob
 català                978       9.35%
 independentista      2691       3.08%
 metge                7541       2.96%
 gai                 16472       2.19%
 historiador         16735       1.61%
 bomber              40199       1.57%
 paleta              32944       1.44%
 filòleg             31196       1.42%
 jueu                14693       1.36%
 alemany              6363       1.16%
 andalús             19049       1.13%
 mort                 1817       1.08%
 músic               10924       1.07%
 periodista           4740       1.04%
 valencià             3236       0.94%
 francès              3807     

### 6. Paraules i subparaules

In [39]:
for i in tokenizer.encode("infermera infermer Civilització urbanitzar urbanització Urbanització"):
    print(i, tokenizer.decode(i))

0 <s>
26849  infermera
4139  infer
4653 mer
3591  Civil
1741 ització
48512  urbanitzar
13830  urbanització
18870  Urban
1741 ització
2 </s>
